In [3]:
import torch
import torch.nn as nn
import sys
sys.path.append('../')
import pandas as pd


from utils.GetData import Data2

In [10]:
PATH = '../in/train.csv'
df = pd.read_csv(PATH)

In [11]:
data = Data2()

In [12]:
df = data.get_data2(df)

AttributeError: 'Data2' object has no attribute 'oh_encoder'